# Desafio

* Utilizando a bibliteca faker crie 1000 registro com o método fake.profile()
* Normalize as respostas em uma dataframe
* Crie a coluna idade: "2023" - "ano de nascimento"
* Com base na idade validade quantas pessoas estão dentro da população ativa (entre 18 anos e 65 anos)
* Valide os emails utilizando o regex visto em aula

In [ ]:
from datetime import datetime
from faker import Faker
import pandas as pd
import great_expectations as gx
fake = Faker('pt_BR')

In [ ]:
profile_df = pd.json_normalize([fake.profile() for i in range(1000)])

In [ ]:
profile_df["today_year"] = datetime.now().date().year

In [ ]:
datetime.now().date().year

In [ ]:
profile_df["today_year"]

In [ ]:
profile_df["age"] = profile_df["today_year"] - pd.to_datetime(profile_df["birthdate"]).dt.year

In [ ]:
profile_df_gx = gx.dataset.PandasDataset(profile_df)

In [ ]:
result = profile_df_gx.expect_column_values_to_be_between(column='age', min_value=18, max_value=65)
result

In [ ]:
result_email = profile_df_gx.expect_column_values_to_match_regex(column='mail', regex='^[\w+.]+@\w+\.\w{2,}(?:\.\w{2})?$')
result_email

# Aula 06

![Great Expectation](https://docs.greatexpectations.io/assets/images/data_context_does_for_you-df2eca32d0152ead16cccd5d3d226abb.png)

## Instanciando um Contexto de Dados

Obteremos um objeto DataContext com o seguinte código:

In [ ]:
import great_expectations as gx

context = gx.get_context()

## Verificando o conteúdo do contexto de dados
Podemos garantir que o Data Context foi instanciado corretamente imprimindo seu conteúdo.

In [ ]:
context

### Salvando o contexto de dados para uso futuro (rodar apenas uma vez)
Como não inicializamos anteriormente um Contexto de dados do sistema de arquivos ou especificamos um caminho para criar um, o Contexto de dados que recebemos de gx.get_context() era um Contexto de dados efêmero temporário na memória. 

Para salvar este Contexto de Dados para uso futuro, vamos convertê-lo em um Contexto de Dados do Sistema de Arquivos:

Você pode fornecer o caminho para uma pasta específica ao converter seu contexto de dados efêmeros em um contexto de dados do sistema de arquivos.
Se você fizer isso, seu contexto de dados do sistema de arquivos será inicializado nesse local. 
Caso contrário, seu novo contexto de dados do sistema de arquivos será inicializado na pasta em que seu script é executado.

In [ ]:
context.convert_to_file_context()

## 2. Conectando aos dados
Nos conectaremos aos dados .csv armazenados em nosso repositório:

In [ ]:
validator = context.sources.pandas_default.read_csv(
    'dados/atividade1/us_tax_data_2016.csv'
)

In [ ]:
validator.head()


## 3. Criando Expectativas

Quando lemos nossos dados .csv, recebemos uma instância do __Validator__ de volta. 

Um __Validador__ é um objeto capaz de armazenar Expectativas sobre os dados aos quais está associado, bem como realizar introspecções sobre esses dados.

A seguir definiremos algumas Expectations.

Com a Expectation definida abaixo, estamos afirmando que esperamos que a coluna ``agi_stub`` seja sempre preenchida. 

Ou seja: nenhum dos valores da coluna deve ser nulo.

In [ ]:
validator.expect_column_values_to_not_be_null('agi_stub')

## 4. Validando os dados

### 4.1 Executando as Expectativas definidas
Agora que definimos nossas expectativas, é hora do GX fazer uma introspecção de nossos dados e ver se eles correspondem ao que dissemos a GX para esperar. Para isso, definimos um Checkpoint (que nos permitirá repetir a Validação no futuro).

In [ ]:
checkpoint = gx.checkpoint.SimpleCheckpoint(
    name = "my_quickstart_checkpoint",
    data_context = context,
    validator = validator
)

Uma vez criado o Checkpoint, vamos executá-lo e receber de volta os resultados da nossa Validação.

In [ ]:
checkpoint_result = checkpoint.run()

In [ ]:
checkpoint_result

### 4.2 Visualizando os resultados
O Great Expectations fornece uma maneira amigável e legível por humanos de visualizar os resultados de Validations: Data Docs. 

Nosso Checkpoint terá compilado automaticamente novos Data Docs para incluir os resultados da validação que executamos, para que possamos visualizá-los imediatamente:

In [ ]:
context.build_data_docs()

In [ ]:
context.open_data_docs()

In [ ]:
context.convert_to_file_context()